# 003_Llam3 실습


## 패키지 설치하기
   - accelerate 패키지: Hugging Face 에서 제공하는 패키지로 PyTorch 모델을 간략하고 효율적으로 처리할 수 있음.*이탤릭체 텍스트*


In [ ]:
!pip install accelerate

## 토큰 설정하기

In [ ]:
import os
os.environ['HF_TOKEN'] = "발급 받은 토큰을 입력 하기"


## 패키지 불러오기

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch


## 토큰 및 모델 다운로드하기


In [ ]:
model_id = "meta-llama/Meta-Llama-3-8B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
        model_id,
        torch_dtype=torch.bfloat16,
        device_map="auto",)

## LLM 응답 함수

In [ ]:
def generate_response(system_message, user_message):
  messages = [
    {"role": "system", "content": system_message},
    {"role": "user", "content": user_message},
  ]

  input_ids = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=True,
    return_tensors="pt").to(model.device)

  terminators = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<|eot_id|>")
  ]

  outputs = model.generate(
    input_ids,
    max_new_tokens=512,
    eos_token_id=terminators,
    do_sample=True,
    temperature=0.6,
    top_p=0.9  )

  response = outputs[0][input_ids.shape[-1]:]
  return tokenizer.decode(response, skip_special_tokens=True)

## LLM 응답 함수 테스트 하기

In [ ]:
!wget https://raw.githubusercontent.com/jskimn/aiservice_lesson/main/003_llama3/NvidiaSmiMonitor.py
from NvidiaSmiMonitor import NvidiaSmiMonitor
import sys
import time

monitor = NvidiaSmiMonitor(interval=60)  # 2초마다 실행
monitor.start()

# 10초 동안 실행 후 종료
#time.sleep(10)


In [ ]:
response = generate_response(system_message="",
                              user_message="한국어로 대답해줘. 배화여대에 대해서 말해줘")
print(response)


monitor.stop()
monitor.join()
print("monitor---------종료")